In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = list(set(stopwords.words('english')))
import contractions
from nltk.stem import PorterStemmer
ps=PorterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91946\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91946\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def preprocess(data):
    processed=[]
    for i in range(data.shape[0]):
        cleaned = re.sub(r'[^(a-zA-Z)\s]','', data['reviews'][i])# remove punctuations 
        tokenized = word_tokenize(cleaned)# tokenize 
        expanded_words=[contractions.fix(w) for w in tokenized]
        stopped_stem = [ps.stem(word) for word in stopped]#stemming
        #lemt = [wordnet.lemmatize(word) for word in expanded_words]#lemmatization
        stopped = [w.lower() for w in expanded_words if not w.lower() in stop_words]# remove stopwords
        s=" "
        s=s.join(stopped)
        processed.append(s)
    data['reviews']=processed

In [4]:
#TRAINING DATA

#positive
files_pos=os.listdir('data/train/pos')
temp=[]
for file in files_pos:
    with open('data/train/pos/'+file, 'r',encoding="utf8") as f:
        temp.append(f.readlines()[0])       
ones=[]
for i in temp:
    ones.append(1)
pos = pd.DataFrame({"reviews": temp,"labels": ones})

#negative
files_pos=os.listdir('data/train/neg')
temp=[]
for file in files_pos:
    with open('data/train/neg/'+file, 'r',encoding="utf8") as f:
        temp.append(f.readlines()[0])       
zeros=[]
for i in temp:
    zeros.append(0)
neg = pd.DataFrame({"reviews": temp,"labels": zeros})

#combined
df=pos.append(neg,ignore_index=True)
df

,reviews,labels
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1
...,...,...
24995,"Towards the end of the movie, I felt it was to...",0
24996,This is the kind of movie that my enemies cont...,0
24997,I saw 'Descent' last night at the Stockholm Fi...,0
24998,Some films that you pick up for a pound turn o...,0


In [5]:
df['reviews'][0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [6]:
preprocess(df)
df.head()

,reviews,labels
0,bromwell high cartoon comedy ran time programs...,1
1,homelessness ( houselessness george carlin sta...,1
2,brilliant overacting lesley ann warren best dr...,1
3,easily underrated film inn brooks cannon sure ...,1
4,typical mel brooks film much less slapstick mo...,1


In [7]:
df['reviews'][0]

'bromwell high cartoon comedy ran time programs school life teachers years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector i am sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity is not'

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()
docs=np.array(df['reviews'])
bag=count.fit_transform(docs)
#print(count.vocabulary_)
print(len(count.vocabulary_))

114895


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [10]:
y_train=df.labels.values
X_train=tfidf.fit_transform(df.reviews)

In [11]:
#TESTING DATA

#positive
files_pos=os.listdir('data/test/pos')
temp=[]
for file in files_pos:
    with open('data/test/pos/'+file, 'r',encoding="utf8") as f:
        temp.append(f.readlines()[0])       
ones=[]
for i in temp:
    ones.append(1)
pos = pd.DataFrame({"reviews": temp,"labels": ones})

#negative
files_pos=os.listdir('data/test/neg')
temp=[]
for file in files_pos:
    with open('data/test/neg/'+file, 'r',encoding="utf8") as f:
        temp.append(f.readlines()[0])       
zeros=[]
for i in temp:
    zeros.append(0)
neg = pd.DataFrame({"reviews": temp,"labels": zeros})

#combined
fd=pos.append(neg,ignore_index=True)
fd

,reviews,labels
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
24995,I occasionally let my kids watch this garbage ...,0
24996,When all we have anymore is pretty much realit...,0
24997,The basic genre is a thriller intercut with an...,0
24998,Four things intrigued me as to this film - fir...,0


In [12]:
fd['reviews'][0]

"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."

In [13]:
preprocess(fd)
fd.head()

,reviews,labels
0,went saw movie last night coaxed friends mine ...,1
1,actor turned director bill paxton follows prom...,1
2,recreational golfer knowledge sports history p...,1
3,saw film sneak preview delightful cinematograp...,1
4,bill paxton taken true story us golf open made...,1


In [14]:
fd['reviews'][0]

'went saw movie last night coaxed friends mine ill admit reluctant see knew ashton kutcher able comedy wrong kutcher played character jake fischer well kevin costner played ben randall professionalism sign good movie toy emotions one exactly entire theater ( sold ) overcome laughter first half movie moved tears second half exiting theater saw many women tears many full grown men well trying desperately let anyone see crying movie great suggest go see judge'

In [15]:
count_test=CountVectorizer()
docs_test=np.array(fd['reviews'])
bag_test=count_test.fit_transform(docs_test)
#print(count_test.vocabulary_)
print(len(count_test.vocabulary_))

112946


In [16]:
y_test=fd.labels.values
X_test=tfidf.transform(fd.reviews)

In [17]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(max_iter=120000)
logmodel.fit(X_train,y_train)

LogisticRegression(max_iter=120000)

In [18]:
from sklearn.naive_bayes import MultinomialNB
MNB=MultinomialNB()
MNB.fit(X_train,y_train)

MultinomialNB()

In [19]:
predictions=logmodel.predict(X_test)
mpred=MNB.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score
print("LOGISTIC MODEL")
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))
print("accuracy score :",accuracy_score(y_test,predictions))
print("f1_score : ",f1_score(y_test,predictions))

LOGISTIC MODEL
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

[[11020  1480]
 [ 1466 11034]]
accuracy score : 0.88216
f1_score :  0.882225953466059


In [21]:
print("MNB")
print(classification_report(y_test,mpred))
print(confusion_matrix(y_test,mpred))
print("accuracy score :",accuracy_score(y_test,mpred))
print("f1_score : ",f1_score(y_test,mpred))

MNB
              precision    recall  f1-score   support

           0       0.80      0.88      0.84     12500
           1       0.87      0.78      0.82     12500

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000

[[11045  1455]
 [ 2711  9789]]
accuracy score : 0.83336
f1_score :  0.8245451482479785
